In [1]:
#| default_exp py2pyi

# Create delegated pyi

## Setup

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
#| export
import ast, sys, inspect, re, os, importlib.util, importlib.machinery

from ast import parse, unparse
from inspect import signature, getsource
from fastcore.utils import *
from fastcore.meta import delegates

## Basics

In [4]:
#| export
def imp_mod(module_path, package=None): # convert py file to module
    "Import dynamically the module referenced in `fn`"
    module_path = str(module_path)
    module_name = os.path.splitext(os.path.basename(module_path))[0]
    spec = importlib.machinery.ModuleSpec(module_name, None, origin=module_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader = importlib.machinery.SourceFileLoader(module_name, module_path)
    if package is not None: module.__package__ = package
    module.__file__ = os.path.abspath(module_path)
    spec.loader.exec_module(module)
    return module

In [5]:
fn = Path('test_py2pyi.py')

In [6]:
mod = imp_mod(fn) 
mod
a = mod.A()
a.h()
a.k() # patch method
a.m() # patch method
a.n() # patch method

<module 'test_py2pyi' (test_py2pyi.py)>

1

1

1

1

In [7]:
#| export
def _get_tree(mod): # convert a module object to AST tree, 
    return parse(getsource(mod))

In [8]:
tree = _get_tree(mod) # and at this point mod and tree are identical in terms of content of the source code ✨✨✨✨✨✨✨

tree # at first, tree is a ast.Module object, can not be viewed properly

In [9]:
print(unparse(tree)) # to see this ast.Module object, we need to make it a string ✨✨✨✨

__all__ = ['f', 'g']
from fastcore.meta import delegates
from fastcore.utils import patch

class X(int):
    pass

def f(a: int, b: str='a') -> str:
    """I am f"""
    return 1

@delegates(f)
def g(c, d: X, **kwargs) -> str:
    """I am g"""
    return 2

def j(c: int, d: str='a') -> str:
    """I am j"""
    return 1

class A:

    @delegates(j)
    def h(self, b: bool=False, **kwargs):
        a = 1
        return a

class B:
    ...

@patch
@delegates(j)
def k(self: (A, B), b: bool=False, **kwargs):
    return 1

@patch
@delegates(j)
def m(self: A, b: bool=False, **kwargs):
    return 1

@patch
def n(self: A, b: bool=False, **kwargs):
    """No delegates here mmm'k?"""
    return 1


In [10]:
#| export
@patch
def __repr__(self:ast.AST): # now, run a cell with `tree` will show the tree properly
    return unparse(self)

@patch
def _repr_markdown_(self:ast.AST):
    return f"""```python
{self!r}
```"""

In [11]:
tree # now, tree can be viewed properly

```python
__all__ = ['f', 'g']
from fastcore.meta import delegates
from fastcore.utils import patch

class X(int):
    pass

def f(a: int, b: str='a') -> str:
    """I am f"""
    return 1

@delegates(f)
def g(c, d: X, **kwargs) -> str:
    """I am g"""
    return 2

def j(c: int, d: str='a') -> str:
    """I am j"""
    return 1

class A:

    @delegates(j)
    def h(self, b: bool=False, **kwargs):
        a = 1
        return a

class B:
    ...

@patch
@delegates(j)
def k(self: (A, B), b: bool=False, **kwargs):
    return 1

@patch
@delegates(j)
def m(self: A, b: bool=False, **kwargs):
    return 1

@patch
def n(self: A, b: bool=False, **kwargs):
    """No delegates here mmm'k?"""
    return 1
```

In [12]:
tree._repr_markdown_() 

'```python\n__all__ = [\'f\', \'g\']\nfrom fastcore.meta import delegates\nfrom fastcore.utils import patch\n\nclass X(int):\n    pass\n\ndef f(a: int, b: str=\'a\') -> str:\n    """I am f"""\n    return 1\n\n@delegates(f)\ndef g(c, d: X, **kwargs) -> str:\n    """I am g"""\n    return 2\n\ndef j(c: int, d: str=\'a\') -> str:\n    """I am j"""\n    return 1\n\nclass A:\n\n    @delegates(j)\n    def h(self, b: bool=False, **kwargs):\n        a = 1\n        return a\n\nclass B:\n    ...\n\n@patch\n@delegates(j)\ndef k(self: (A, B), b: bool=False, **kwargs):\n    return 1\n\n@patch\n@delegates(j)\ndef m(self: A, b: bool=False, **kwargs):\n    return 1\n\n@patch\ndef n(self: A, b: bool=False, **kwargs):\n    """No delegates here mmm\'k?"""\n    return 1\n```'

In [13]:
for idx, o in enumerate(tree.body): 
    print(f'node: {idx}, type: {type(o)}') # let check each part/node of the tree and their real types ✨✨✨✨
    o

node: 0, type: <class 'ast.Assign'>


```python
__all__ = ['f', 'g']
```

node: 1, type: <class 'ast.ImportFrom'>


```python
from fastcore.meta import delegates
```

node: 2, type: <class 'ast.ImportFrom'>


...

```python
@patch
@delegates(j)
def k(self: (A, B), b: bool=False, **kwargs):
    return 1
```

node: 10, type: <class 'ast.FunctionDef'>


```python
@patch
@delegates(j)
def m(self: A, b: bool=False, **kwargs):
    return 1
```

node: 11, type: <class 'ast.FunctionDef'>


```python
@patch
def n(self: A, b: bool=False, **kwargs):
    """No delegates here mmm'k?"""
    return 1
```

In [14]:
node_func = tree.body[4] # yes, Jeremy make each item as a node ✨✨✨✨
node_func 

```python
def f(a: int, b: str='a') -> str:
    """I am f"""
    return 1
```

In [15]:
#| export
functypes = (ast.FunctionDef,ast.AsyncFunctionDef) # consider both normal func and async func as functypes

In [16]:
isinstance(node_func, functypes)

True

In [17]:
#| export
def _deco_id(d:Union[ast.Name,ast.Attribute])->bool:
    "Get the id for AST node `d`"
    return d.id if isinstance(d, ast.Name) else d.func.id

def has_deco(node:Union[ast.FunctionDef,ast.AsyncFunctionDef], name:str)->bool:
    "Check if a function node `node` has a decorator named `name`"
    return any(_deco_id(d)==name for d in getattr(node, 'decorator_list', []))

# check whether a decorator by its name is stored inside the node's decorator list ✨✨✨✨

In [18]:
nm = 'delegates'
has_deco(node_func, nm)

False

In [19]:
node_del = tree.body[5]
node_del
nm = 'delegates'
has_deco(node_del, nm)

```python
@delegates(f)
def g(c, d: X, **kwargs) -> str:
    """I am g"""
    return 2
```

True

In [20]:
node_pa = tree.body[9]
node_pa
node_pa2 = tree.body[10]
node_pa3 = tree.body[11]
node_pa2
node_pa3

```python
@patch
@delegates(j)
def k(self: (A, B), b: bool=False, **kwargs):
    return 1
```

```python
@patch
@delegates(j)
def m(self: A, b: bool=False, **kwargs):
    return 1
```

```python
@patch
def n(self: A, b: bool=False, **kwargs):
    """No delegates here mmm'k?"""
    return 1
```

In [21]:
has_deco(node_del, 'delegates'), has_deco(node_pa, 'patch')

(True, True)

In [22]:
node_import = tree.body[1]
node_import
node_class = tree.body[3]
node_class
node_classA = tree.body[7]
node_classB = tree.body[8]
node_classA
node_classB

```python
from fastcore.meta import delegates
```

```python
class X(int):
    pass
```

```python
class A:

    @delegates(j)
    def h(self, b: bool=False, **kwargs):
        a = 1
        return a
```

```python
class B:
    ...
```

## Function processing

In [23]:
def _proc_body   (node, mod): print('_proc_body', type(node), node)
def _proc_func   (node, mod): print('_proc_func', type(node), node)
def _proc_class  (node, mod): print('_proc_class', type(node), node)
def _proc_patched(node, mod): print('_proc_patched', type(node), node)

# check each node and give proper processing function to each node ✨✨✨✨

In [24]:


_proc_body(node_import, mod)
_proc_body(node_func, mod)
_proc_func(node_del, mod)
_proc_func(node_pa, mod)
_proc_class(node_class, mod)
_proc_patched(node_pa, mod)

_proc_body <class 'ast.ImportFrom'> from fastcore.meta import delegates
_proc_body <class 'ast.FunctionDef'> def f(a: int, b: str='a') -> str:
    """I am f"""
    return 1
_proc_func <class 'ast.FunctionDef'> @delegates(f)
def g(c, d: X, **kwargs) -> str:
    """I am g"""
    return 2
_proc_func <class 'ast.FunctionDef'> @patch
@delegates(j)
def k(self: (A, B), b: bool=False, **kwargs):
    return 1
_proc_class <class 'ast.ClassDef'> class X(int):
    pass
_proc_patched <class 'ast.FunctionDef'> @patch
@delegates(j)
def k(self: (A, B), b: bool=False, **kwargs):
    return 1


In [25]:
#| export
def _get_proc(node):
    # if node is a class type, use _proc_class
    if isinstance(node, ast.ClassDef): return _proc_class
    # if node is not a function type, return None
    if not isinstance(node, functypes): return None
    # if node has not a decorator named 'delegates', return _proc_body
    if not has_deco(node, 'delegates'): return _proc_body
    # if node has a decorator named 'patch', return _proc_patched
    if has_deco(node, 'patch'): return _proc_patched
    # all other cases, return _proc_func
    return _proc_func

In [26]:
_get_proc(node_import) == None
_get_proc(node_func) == _proc_body
_get_proc(node_del) == _proc_func
_get_proc(node_pa) == _proc_patched
_get_proc(node_class) == _proc_class

True

True

True

True

True

In [27]:
#| export
def _proc_tree(tree, mod):
    for node in tree.body:
        proc = _get_proc(node)
        if proc: proc(node, mod) # eventually bring node of AST and mod of runtime together ✨✨✨✨

# check each node of a tree and decide which processing function to use, and use it if available ✨✨✨✨

In [28]:
#| export
def _proc_mod(mod):
    tree = _get_tree(mod) # get AST tree from runtime mod
    _proc_tree(tree, mod) # check every node of the tree, run the proper proc for each node
    return tree

# turn a module object to AST tree, and check each node of the tree, decide which node get a processing function to run, and then run the proper proc for the node ✨✨✨✨


In [29]:
_proc_mod(mod)
# as we can see below, the first 3 lines or nodes are ignored or without the processing functions ✨✨✨✨

_proc_class <class 'ast.ClassDef'> class X(int):
    pass
_proc_body <class 'ast.FunctionDef'> def f(a: int, b: str='a') -> str:
    """I am f"""
    return 1
_proc_func <class 'ast.FunctionDef'> @delegates(f)
def g(c, d: X, **kwargs) -> str:
    """I am g"""
    return 2
_proc_body <class 'ast.FunctionDef'> def j(c: int, d: str='a') -> str:
    """I am j"""
    return 1
_proc_class <class 'ast.ClassDef'> class A:

    @delegates(j)
    def h(self, b: bool=False, **kwargs):
        a = 1
        return a
_proc_class <class 'ast.ClassDef'> class B:
    ...
_proc_patched <class 'ast.FunctionDef'> @patch
@delegates(j)
def k(self: (A, B), b: bool=False, **kwargs):
    return 1
_proc_patched <class 'ast.FunctionDef'> @patch
@delegates(j)
def m(self: A, b: bool=False, **kwargs):
    return 1
_proc_body <class 'ast.FunctionDef'> @patch
def n(self: A, b: bool=False, **kwargs):
    """No delegates here mmm'k?"""
    return 1


```python
__all__ = ['f', 'g']
from fastcore.meta import delegates
from fastcore.utils import patch

class X(int):
    pass

def f(a: int, b: str='a') -> str:
    """I am f"""
    return 1

@delegates(f)
def g(c, d: X, **kwargs) -> str:
    """I am g"""
    return 2

def j(c: int, d: str='a') -> str:
    """I am j"""
    return 1

class A:

    @delegates(j)
    def h(self, b: bool=False, **kwargs):
        a = 1
        return a

class B:
    ...

@patch
@delegates(j)
def k(self: (A, B), b: bool=False, **kwargs):
    return 1

@patch
@delegates(j)
def m(self: A, b: bool=False, **kwargs):
    return 1

@patch
def n(self: A, b: bool=False, **kwargs):
    """No delegates here mmm'k?"""
    return 1
```

In [30]:
node_func.name
node_del.name
node_pa.name
node_class.name
node_import.names 
node_import.names[0].name

'f'

'g'

'k'

'X'

[delegates]

'delegates'

### Get symbol from module 

- use `mod` module object and `name` str to get symbol object from module
- but we can't get the symbol for methods, delegates or patch, with `mod`
- symbol (in the runtime) has more and accurate info than node (not in runtime), though both are derived from module object ✨✨✨✨✨

In [31]:
sym_f = getattr(mod, node_func.name) # sym is actually func or class like getattr(x, 'y') is to get x.y 
node_func, node_func.name, sym_f # , type(sym_f)

sym_g = getattr(mod, node_del.name)
node_del, node_del.name, sym_g

sym_k = getattr(mod, node_pa.name) # 🔥🔥🔥🔥🔥 pylance's getattr can't get patch method from mod the runtime ✨✨✨✨
node_pa, node_pa.name, sym_k

sym_m = getattr(mod, node_pa2.name)
node_pa2, node_pa2.name, sym_m

sym_n = getattr(mod, node_pa3.name) # 
node_pa3, node_pa3.name, sym_n # the output of sym is "ge", this is very strange ✨✨✨✨ ?????

sym_class = getattr(mod, node_class.name)
node_class, node_class.name, sym_class

sym_classA = getattr(mod, node_classA.name)
node_classA, node_classA.name, sym_classA

sym_classB = getattr(mod, node_classB.name)
node_classB, node_classB.name, sym_classB # only return the function object ???? ✨✨✨✨


(def f(a: int, b: str='a') -> str:
     """I am f"""
     return 1,
 'f',
 <function test_py2pyi.f(a: int, b: str = 'a') -> str>)

(@delegates(f)
 def g(c, d: X, **kwargs) -> str:
     """I am g"""
     return 2,
 'g',
 <function test_py2pyi.g(c, d: test_py2pyi.X, *, b: str = 'a') -> str>)

(@patch
 @delegates(j)
 def k(self: (A, B), b: bool=False, **kwargs):
     return 1,
 'k',
 None)

(@patch
 @delegates(j)
 def m(self: A, b: bool=False, **kwargs):
     return 1,
 'm',
 None)

(@patch
 def n(self: A, b: bool=False, **kwargs):
     """No delegates here mmm'k?"""
     return 1,
 'n',
 'ge')

(class X(int):
     pass,
 'X',
 test_py2pyi.X)

(class A:
 
     @delegates(j)
     def h(self, b: bool=False, **kwargs):
         a = 1
         return a,
 'A',
 test_py2pyi.A)

(class B:
     ...,
 'B',
 test_py2pyi.B)

but we can get symbols for methods, delegates or patch, with `mod.A` instead `mod`

In [32]:
# manually change `mod` to `mod.A`: disadvantage is not programmable 🔥🔥🔥🔥🔥
sym_k = getattr(mod.A, node_pa.name) # if mod not working, how about mod.A() ✨✨✨✨ 
node_pa, node_pa.name, sym_k
sym_m = getattr(mod.A, node_pa2.name)
node_pa2, node_pa2.name, sym_m
sym_n = getattr(mod.A, node_pa3.name)
node_pa3, node_pa3.name, sym_n


(@patch
 @delegates(j)
 def k(self: (A, B), b: bool=False, **kwargs):
     return 1,
 'k',
 <function test_py2pyi.A.k(self: (<class 'test_py2pyi.A'>, <class 'test_py2pyi.B'>), b: bool = False, *, d: str = 'a')>)

(@patch
 @delegates(j)
 def m(self: A, b: bool=False, **kwargs):
     return 1,
 'm',
 <function test_py2pyi.A.m(self: test_py2pyi.A, b: bool = False, *, d: str = 'a')>)

(@patch
 def n(self: A, b: bool=False, **kwargs):
     """No delegates here mmm'k?"""
     return 1,
 'n',
 <function test_py2pyi.A.n(self: test_py2pyi.A, b: bool = False, **kwargs)>)

### to make the above mod.A to be programmable, we need sth like mod.__dict__[node_classA] ✨✨✨✨
### please note: symbol has more accurate and more info than node 

In [33]:


classA_name = tree.body[7].name
classA_name
assert mod.A, mod.__dict__[classA_name]

sym_k = getattr(mod.__dict__[classA_name], node_pa.name) # if mod not working, how about mod.A() ✨✨✨✨ 
node_pa, node_pa.name, sym_k
sym_m = getattr(mod.__dict__[classA_name], node_pa2.name)
node_pa2, node_pa2.name, sym_m
sym_n = getattr(mod.__dict__[classA_name], node_pa3.name)
node_pa3, node_pa3.name, sym_n


#### watch out for this part, it's very important ✨✨✨✨
# sym_class = getattr(mod.X, node_class.name)
# node_class, node_class.name, sym_class

# sym_classA = getattr(mod, node_classA.name)
# node_classA, node_classA.name, sym_classA

# sym_classB = getattr(mod, node_classB.name)
# node_classB, node_classB.name, sym_classB


'A'

(@patch
 @delegates(j)
 def k(self: (A, B), b: bool=False, **kwargs):
     return 1,
 'k',
 <function test_py2pyi.A.k(self: (<class 'test_py2pyi.A'>, <class 'test_py2pyi.B'>), b: bool = False, *, d: str = 'a')>)

(@patch
 @delegates(j)
 def m(self: A, b: bool=False, **kwargs):
     return 1,
 'm',
 <function test_py2pyi.A.m(self: test_py2pyi.A, b: bool = False, *, d: str = 'a')>)

(@patch
 def n(self: A, b: bool=False, **kwargs):
     """No delegates here mmm'k?"""
     return 1,
 'n',
 <function test_py2pyi.A.n(self: test_py2pyi.A, b: bool = False, **kwargs)>)

In [34]:
sig_f = signature(sym_f)
print(sig_f)

sig_g = signature(sym_g)
print(sig_g)

sig_k = signature(sym_k)
print(sig_k)

sig_m = signature(sym_m)
print(sig_m)

sig_n = signature(sym_n)
print(sig_n)

(a: int, b: str = 'a') -> str
(c, d: test_py2pyi.X, *, b: str = 'a') -> str
(self: (<class 'test_py2pyi.A'>, <class 'test_py2pyi.B'>), b: bool = False, *, d: str = 'a')
(self: test_py2pyi.A, b: bool = False, *, d: str = 'a')
(self: test_py2pyi.A, b: bool = False, **kwargs)


In [35]:
#| export
def sig2str(sig):
    s = str(sig)
    s = re.sub(r"<class '(.*?)'>", r'\1', s)
    s = re.sub(r"dynamic_module\.", "", s)
    return s

In [36]:
#| export
def ast_args(func):
    sig = signature(func)
    return ast.parse(f"def _{sig2str(sig)}: ...").body[0].args

### node vs node.args vs ast_args

In [37]:
node_func, node_func.args, ast_args(sym_f) # same
node_del, node_del.args, ast_args(sym_g) # symbol show advanced type hinting

node_pa, node_pa.args, ast_args(sym_k) # symbol show advanced type hinting
node_pa2, node_pa2.args, ast_args(sym_m) # symbol show advanced type hinting
node_pa3, node_pa3.args, ast_args(sym_n)


(def f(a: int, b: str='a') -> str:
     """I am f"""
     return 1,
 a: int, b: str='a',
 a: int, b: str='a')

(@delegates(f)
 def g(c, d: X, **kwargs) -> str:
     """I am g"""
     return 2,
 c, d: X, **kwargs,
 c, d: test_py2pyi.X, *, b: str='a')

(@patch
 @delegates(j)
 def k(self: (A, B), b: bool=False, **kwargs):
     return 1,
 self: (A, B), b: bool=False, **kwargs,
 self: (test_py2pyi.A, test_py2pyi.B), b: bool=False, *, d: str='a')

(@patch
 @delegates(j)
 def m(self: A, b: bool=False, **kwargs):
     return 1,
 self: A, b: bool=False, **kwargs,
 self: test_py2pyi.A, b: bool=False, *, d: str='a')

(@patch
 def n(self: A, b: bool=False, **kwargs):
     """No delegates here mmm'k?"""
     return 1,
 self: A, b: bool=False, **kwargs,
 self: test_py2pyi.A, b: bool=False, **kwargs)

### `_body_ellip`: keep node's docstring and remove its content

In [38]:
#| export
def _body_ellip(n: ast.AST): # only accept AST node
    # keep the docstring if it's available
    stidx = 1 if isinstance(n.body[0], ast.Expr) and isinstance(n.body[0].value, ast.Str) else 0
    # replace the body with ...
    n.body[stidx:] = [ast.Expr(ast.Constant(...))]

In [39]:
node_func
_body_ellip(node_func)
node_func

```python
def f(a: int, b: str='a') -> str:
    """I am f"""
    return 1
```

```python
def f(a: int, b: str='a') -> str:
    """I am f"""
    ...
```

### `_update_func`: update each func/node 
- give advanced hinting symbols to each node
- keep docstring and remove body content
- remove delegates from decorator list for the node

In [40]:
node_func.decorator_list # each node of AST keeps a decorator_list ✨✨✨✨
node_class.decorator_list
node_del.decorator_list
node_pa.decorator_list
node_pa2.decorator_list
node_pa3.decorator_list

[]

[]

[delegates(f)]

[patch, delegates(j)]

[patch, delegates(j)]

[patch]

In [41]:
#| export
def _update_func(node, sym):
    """Replace the parameter list of the source code of a function `f` with a different signature.
    Replace the body of the function with just `pass`, and remove any decorators named 'delegates'"""
    # give args in the form of mod to node of AST
    node.args = ast_args(sym)
    # keep the node's doc and replace body with ...
    _body_ellip(node)
    # remove delegates decorator from the node's decorator list
    node.decorator_list = [d for d in node.decorator_list if _deco_id(d) != 'delegates']

In [42]:
node_func
_update_func(node_func, sym_f)
node_func

# node_class.decorator_list ## to watch out for this part ✨✨✨✨
# _update_func(node_class, sym_class)


node_del
_update_func(node_del, sym_g)
node_del

node_pa
_update_func(node_pa, sym_k)
node_pa

node_pa2
_update_func(node_pa2, sym_m)
node_pa2

node_pa3
_update_func(node_pa3, sym_n)
node_pa3

```python
def f(a: int, b: str='a') -> str:
    """I am f"""
    ...
```

```python
def f(a: int, b: str='a') -> str:
    """I am f"""
    ...
```

```python
@delegates(f)
def g(c, d: X, **kwargs) -> str:
    """I am g"""
    return 2
```

```python
def g(c, d: test_py2pyi.X, *, b: str='a') -> str:
    """I am g"""
    ...
```

```python
@patch
@delegates(j)
def k(self: (A, B), b: bool=False, **kwargs):
    return 1
```

```python
@patch
def k(self: (test_py2pyi.A, test_py2pyi.B), b: bool=False, *, d: str='a'):
    ...
```

```python
@patch
@delegates(j)
def m(self: A, b: bool=False, **kwargs):
    return 1
```

```python
@patch
def m(self: test_py2pyi.A, b: bool=False, *, d: str='a'):
    ...
```

```python
@patch
def n(self: A, b: bool=False, **kwargs):
    """No delegates here mmm'k?"""
    return 1
```

```python
@patch
def n(self: test_py2pyi.A, b: bool=False, **kwargs):
    """No delegates here mmm'k?"""
    ...
```

### update all the processing functions

In [43]:
#| export
def _proc_body(node, mod): _body_ellip(node)
# update _proc_body with _body_ellip ✨✨✨✨
# keep doc and replace body of the node with ...

In [44]:
#| export
def _proc_func(node, mod): 
    sym = getattr(mod, node.name)
    _update_func(node, sym)

# update _proc_func with _update_func ✨✨✨✨
# to all normal functions and async functions
# - update advanced type hinting
# - keep doc and replace body of the node with ...
# - remove delegates decorator from the node's decorator list

In [45]:
_proc_mod??


Signature: _proc_mod(mod)
Docstring: <no docstring>
Source:   
def _proc_mod(mod):
    tree = _get_tree(mod) # get AST tree from runtime mod
    _proc_tree(tree, mod) # check every node of the tree, run the proper proc for each node
    return tree
File:      /var/folders/gz/ch3n2mp51m9386sytqf97s6w0000gn/T/ipykernel_72664/1865813169.py
Type:      function

In [46]:
_proc_tree??

Signature: _proc_tree(tree, mod)
Docstring: <no docstring>
Source:   
def _proc_tree(tree, mod):
    for node in tree.body:
        proc = _get_proc(node)
        if proc: proc(node, mod) # eventually bring node of AST and mod of runtime together ✨✨✨✨
File:      /var/folders/gz/ch3n2mp51m9386sytqf97s6w0000gn/T/ipykernel_72664/3378100334.py
Type:      function

In [47]:
tree = _get_tree(mod)
tree
tree.body[4] # normal func without delegates or patch decorator
tree = _proc_mod(mod)
tree.body[4]

```python
__all__ = ['f', 'g']
from fastcore.meta import delegates
from fastcore.utils import patch

class X(int):
    pass

def f(a: int, b: str='a') -> str:
    """I am f"""
    return 1

@delegates(f)
def g(c, d: X, **kwargs) -> str:
    """I am g"""
    return 2

def j(c: int, d: str='a') -> str:
    """I am j"""
    return 1

class A:

    @delegates(j)
    def h(self, b: bool=False, **kwargs):
        a = 1
        return a

class B:
    ...

@patch
@delegates(j)
def k(self: (A, B), b: bool=False, **kwargs):
    return 1

@patch
@delegates(j)
def m(self: A, b: bool=False, **kwargs):
    return 1

@patch
def n(self: A, b: bool=False, **kwargs):
    """No delegates here mmm'k?"""
    return 1
```

```python
def f(a: int, b: str='a') -> str:
    """I am f"""
    return 1
```

_proc_class <class 'ast.ClassDef'> class X(int):
    pass
_proc_class <class 'ast.ClassDef'> class A:

    @delegates(j)
    def h(self, b: bool=False, **kwargs):
        a = 1
        return a
_proc_class <class 'ast.ClassDef'> class B:
    ...
_proc_patched <class 'ast.FunctionDef'> @patch
@delegates(j)
def k(self: (A, B), b: bool=False, **kwargs):
    return 1
_proc_patched <class 'ast.FunctionDef'> @patch
@delegates(j)
def m(self: A, b: bool=False, **kwargs):
    return 1


```python
def f(a: int, b: str='a') -> str:
    """I am f"""
    ...
```

## Patch

In [48]:
tree

```python
__all__ = ['f', 'g']
from fastcore.meta import delegates
from fastcore.utils import patch

class X(int):
    pass

def f(a: int, b: str='a') -> str:
    """I am f"""
    ...

def g(c, d: test_py2pyi.X, *, b: str='a') -> str:
    """I am g"""
    ...

def j(c: int, d: str='a') -> str:
    """I am j"""
    ...

class A:

    @delegates(j)
    def h(self, b: bool=False, **kwargs):
        a = 1
        return a

class B:
    ...

@patch
@delegates(j)
def k(self: (A, B), b: bool=False, **kwargs):
    return 1

@patch
@delegates(j)
def m(self: A, b: bool=False, **kwargs):
    return 1

@patch
def n(self: A, b: bool=False, **kwargs):
    """No delegates here mmm'k?"""
    ...
```

In [49]:
node = tree.body[9]
node
# but how to extract class A's name programmatically ✨✨✨✨ (see next cell)
"mod.k: ", getattr(mod, node.name)
sym = getattr(mod.__dict__["A"], node.name)
sym
node.args, ast_args(sym)

```python
@patch
@delegates(j)
def k(self: (A, B), b: bool=False, **kwargs):
    return 1
```

('mod.k: ', None)

<function test_py2pyi.A.k(self: (<class 'test_py2pyi.A'>, <class 'test_py2pyi.B'>), b: bool = False, *, d: str = 'a')>

(self: (A, B), b: bool=False, **kwargs,
 self: (test_py2pyi.A, test_py2pyi.B), b: bool=False, *, d: str='a')

In [50]:
node.args, type(node.args)
node.args.args, type(node.args.args) # a list
node.args.args[0]
node.args.args[0].annotation
node.args.args[0].annotation.elts
node.args.args[0].annotation.elts[0], type(node.args.args[0].annotation.elts[0])
className = node.args.args[0].annotation.elts[0]
className.id, type(className.id)

(self: (A, B), b: bool=False, **kwargs, ast.arguments)

([self: (A, B), b: bool], list)

```python
self: (A, B)
```

```python
(A, B)
```

[A, B]

(A, ast.Name)

('A', str)

In [51]:
# make the cell above programmable ✨✨✨✨
ann = node.args.args[0].annotation
if hasattr(ann, 'elts'): ann = ann.elts[0]
nm = ann.id
nm

'A'

### `cls_A` from `getattr(mod, 'A')` contains both h, k, m ✨✨✨✨✨

In [52]:
# tree
node_classA = tree.body[7]
node_classA
"node,", node # k
"nm", nm # A
cls = getattr(mod, nm) # class A
"cls", cls
sym_mod_k = getattr(mod, node.name)
sym_cls_k = getattr(cls, node.name)
# sym_mod_h = getattr(mod, 'h') # error
sym_cls_h = getattr(cls, 'h')
sym_cls_m = getattr(cls, 'm')
sym_cls_n = getattr(cls, 'n')
sym_mod_k, sym_cls_k, sym_cls_h, sym_cls_m, sym_cls_n
ast_args(sym)

```python
class A:

    @delegates(j)
    def h(self, b: bool=False, **kwargs):
        a = 1
        return a
```

('node,',
 @patch
 @delegates(j)
 def k(self: (A, B), b: bool=False, **kwargs):
     return 1)

('nm', 'A')

('cls', test_py2pyi.A)

(None,
 <function test_py2pyi.A.k(self: (<class 'test_py2pyi.A'>, <class 'test_py2pyi.B'>), b: bool = False, *, d: str = 'a')>,
 <function test_py2pyi.A.h(self, b: bool = False, *, d: str = 'a')>,
 <function test_py2pyi.A.m(self: test_py2pyi.A, b: bool = False, *, d: str = 'a')>,
 <function test_py2pyi.A.n(self: test_py2pyi.A, b: bool = False, **kwargs)>)

```python
self: (test_py2pyi.A, test_py2pyi.B), b: bool=False, *, d: str='a'
```

In [53]:
sig2str(signature(sym))

"(self: (test_py2pyi.A, test_py2pyi.B), b: bool = False, *, d: str = 'a')"

In [54]:
tree = _get_tree(mod)
node = tree.body[9]
node
"node.args: ", node.args
"mod.k: ", getattr(mod, node.name)
# "mod.k: ", getattr(mod.k, node.name) # error
"mod.A.k: ", getattr(mod.__dict__["A"], node.name)

```python
@patch
@delegates(j)
def k(self: (A, B), b: bool=False, **kwargs):
    return 1
```

('node.args: ', self: (A, B), b: bool=False, **kwargs)

('mod.k: ', None)

('mod.A.k: ',
 <function test_py2pyi.A.k(self: (<class 'test_py2pyi.A'>, <class 'test_py2pyi.B'>), b: bool = False, *, d: str = 'a')>)

In [55]:
sym = getattr(mod.__dict__["A"], node.name)
node
_update_func(node, sym)
node # delegates removed, body replaced with ...

```python
@patch
@delegates(j)
def k(self: (A, B), b: bool=False, **kwargs):
    return 1
```

```python
@patch
def k(self: (test_py2pyi.A, test_py2pyi.B), b: bool=False, *, d: str='a'):
    ...
```

In [56]:
#| export
def _proc_patched(node, mod):
    # patched method can get its class with 2 lines below
    ann = node.args.args[0].annotation
    if hasattr(ann, 'elts'): ann = ann.elts[0]
    # get a mod class like A in cls
    cls = getattr(mod, ann.id)
    # get a method symbol or function from the class
    sym = getattr(cls, node.name)
    # remove its delegates decorator and update its signature, and replace its body with ...
    _update_func(node, sym)

In [57]:
tree = _get_tree(mod)
node = tree.body[9]
node
_proc_patched(node, mod)
node

```python
@patch
@delegates(j)
def k(self: (A, B), b: bool=False, **kwargs):
    return 1
```

```python
@patch
def k(self: (test_py2pyi.A, test_py2pyi.B), b: bool=False, *, d: str='a'):
    ...
```

In [58]:
# tree = _proc_mod(mod)
# tree.body[9]

## Class and file

In [59]:
tree = _get_tree(mod)
node = tree.body[7]
node

```python
class A:

    @delegates(j)
    def h(self, b: bool=False, **kwargs):
        a = 1
        return a
```

In [60]:
node.name, node.body

for sub_node in node.body:
    sub_node

cls = getattr(mod, node.name)
type(cls.k).__name__

for sub_sym in cls.__dict__.values():
     if type(sub_sym).__name__ == 'function':
          sub_sym
    

('A',
 [@delegates(j)
  def h(self, b: bool=False, **kwargs):
      a = 1
      return a])

```python
@delegates(j)
def h(self, b: bool=False, **kwargs):
    a = 1
    return a
```

'function'

<function test_py2pyi.A.h(self, b: bool = False, *, d: str = 'a')>

<function test_py2pyi.A.k(self: (<class 'test_py2pyi.A'>, <class 'test_py2pyi.B'>), b: bool = False, *, d: str = 'a')>

<function test_py2pyi.A.m(self: test_py2pyi.A, b: bool = False, *, d: str = 'a')>

<function test_py2pyi.A.n(self: test_py2pyi.A, b: bool = False, **kwargs)>

In [61]:
#| export
def _proc_class(node, mod): # redefine _proc_class with real processing function
    # node is a class node
    # get the class symbol from the mod with class name
    cls = getattr(mod, node.name)
    # loop through all nodes of the class
    _proc_tree(node, cls)


# My solution with claude

In [63]:
#| export
def _proc_class(node, mod): # node is a class node, mod is the module
    # Get the runtime class object from the module
    # important difference: runtime class object contains all patched methods, class node does not ✨✨✨✨✨
    cls = getattr(mod, node.name)
    
    # Iterate through all attributes of the class
    for name, member in cls.__dict__.items():
        # Check if the attribute is a callable (method) and not a special method (i.e., doesn't start with __)
        if callable(member) and not name.startswith('__'):
            # Check if this method is not already in the AST node's body (ie., patched method)
            if name not in [n.name for n in node.body if isinstance(n, ast.FunctionDef)]:
                print(name, member)  # Debug print
                # Add this (patched) method to the class AST node
                _add_method_to_class(node, name, member)
    
    # Process other members of the class 
    _proc_tree(node, cls) # dive into the other sub nodes (normal class methods) inside the runtime class object of the module

def _add_method_to_class(class_node, method_name, method_object):
    # Create a new FunctionDef node for the method
    method_ast = ast.FunctionDef(
        name=method_name,
        args=ast_args(method_object),  # Get the arguments of the method
        body=[],  # Empty body for now
        decorator_list=[],  # No decorators
        lineno=1,  # Set line number (arbitrary)
        col_offset=0,  # Set column offset (arbitrary)
        end_lineno=1,  # Set end line number (arbitrary)
        end_col_offset=0,  # Set end column offset (arbitrary)
    )
    
    # Remove annotation for 'self' parameter if it exists
    if method_ast.args.args and method_ast.args.args[0].arg == 'self':
        method_ast.args.args[0].annotation = None
    
    # Add docstring to the method if it exists
    doc = method_object.__doc__
    if doc:
        docstring = ast.Expr(value=ast.Constant(value=doc, kind=None))
        method_ast.body.append(docstring)
    
    # Add ellipsis to the method body
    _body_ellip(method_ast)

    # Remove any existing ellipsis in the class body
    class_node.body = [node for node in class_node.body 
                       if not (isinstance(node, ast.Expr) and 
                               isinstance(node.value, ast.Constant) and 
                               node.value.value is Ellipsis)]
    
    # Add the method to the class body
    class_node.body.append(method_ast)

def _body_ellip(n: ast.AST):
    # Check if there's already an ellipsis in the body
    has_ellipsis = any(
        isinstance(node, ast.Expr) and 
        isinstance(node.value, ast.Constant) and 
        node.value.value is Ellipsis
        for node in n.body
    )
    
    if not has_ellipsis:
        # Keep the docstring if it's available
        stidx = 1 if n.body and isinstance(n.body[0], ast.Expr) and isinstance(n.body[0].value, ast.Constant) else 0
        # Add a single ellipsis, preserving any existing docstring
        n.body[stidx:] = [ast.Expr(ast.Constant(...))]

def _proc_mod(mod):
    # Get the AST for the entire module
    tree = _get_tree(mod)
    # Process the module tree
    _proc_tree(tree, mod)
    # Remove standalone patched functions from the module body
    tree.body = [node for node in tree.body 
                 if not (isinstance(node, ast.FunctionDef) and has_deco(node, 'patch'))]
    return tree

def ast_args(func):
    # Get the signature of the function
    sig = signature(func)
    
    def clean_annotation(param_name, anno): 
        # The lines below are commented out to keep 'self' annotations
        # if param_name == 'self':
        #     return None
        return anno  # Return the annotation as-is for now

    # Create new parameters with cleaned annotations
    cleaned_params = [
        param.replace(annotation=clean_annotation(param.name, param.annotation))
        for param in sig.parameters.values()
    ]
    
    # Create a new signature with the cleaned parameters
    cleaned_sig = sig.replace(parameters=cleaned_params)
    # Convert the signature to an AST arguments node
    return ast.parse(f"def _{sig2str(cleaned_sig)}: ...").body[0].args

def sig2str(sig):
    # Convert the signature to a string
    s = str(sig)
    # Replace class representations with just the class name
    s = re.sub(r"<class '(.*?)'>", r'\1', s)
    # Remove module prefixes from type names
    s = re.sub(r"(\w+\.)+(\w+)", r'\2', s)
    # Remove 'dynamic_module.' prefix
    s = re.sub(r"dynamic_module\.", "", s)
    return s

def _proc_patched(node, mod):
    # This function is now empty as patched methods are handled in _proc_class
    pass


In [64]:
tree = _get_tree(mod)
node = tree.body[7]

node
_proc_class(node, mod)
node 

```python
class A:

    @delegates(j)
    def h(self, b: bool=False, **kwargs):
        a = 1
        return a
```

k <function A.k at 0x11fe29360>
m <function A.m at 0x11fe29480>
n <function A.n at 0x11fe29510>


```python
class A:

    def h(self, b: bool=False, *, d: str='a'):
        ...

    def k(self, b: bool=False, *, d: str='a'):
        ...

    def m(self, b: bool=False, *, d: str='a'):
        ...

    def n(self, b: bool=False, **kwargs):
        """No delegates here mmm'k?"""
        ...
```

In [65]:
#| export
def create_pyi(fn, package=None):
    "Convert `fname.py` to `fname.pyi` by removing function bodies and expanding `delegates` kwargs"
    fn = Path(fn)
    mod = imp_mod(fn, package=package)
    tree = _proc_mod(mod)
    res = unparse(tree)
    fn.with_suffix('.pyi').write_text(res)

In [66]:
create_pyi('test_py2pyi.py')
create_pyi('simple-test.py')

k <function A.k at 0x11fe2b250>
m <function A.m at 0x11fe2b370>
n <function A.n at 0x11fe2b400>
k <function B.k at 0x11fe2b2e0>
f <function A.f at 0x11fe2b520>


In [ ]:
# fn = Path('/Users/jhoward/git/fastcore/fastcore/docments.py')
# create_pyi(fn, 'fastcore')

## Script

In [ ]:
#| export
from fastcore.script import call_parse

In [ ]:
#| export
@call_parse
def py2pyi(fname:str,  # The file name to convert
           package:str=None  # The parent package
          ):
    "Convert `fname.py` to `fname.pyi` by removing function bodies and expanding `delegates` kwargs"
    create_pyi(fname, package)

In [ ]:
#| export
@call_parse
def replace_wildcards(
    # Path to the Python file to process
    path: str):
    "Expand wildcard imports in the specified Python file."
    path = Path(path)
    path.write_text(expand_wildcards(path.read_text()))

# Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()